In [8]:
# Import required libraries
from pyspark.sql import SparkSession
import os
import json
from IPython.display import display, Markdown

# Initialize Spark session
spark = SparkSession.builder.appName("FHIR Parquet README Generator").getOrCreate()

# Set the root directory containing Parquet folders and the output README file path
parquet_root_directory = "/home/jovyan/work/fhir-data/fhir-parquet"
output_file = "/home/jovyan/work/fhir-data/README.md"

def get_field_structure(obj, prefix=''):
    structure = {}
    if isinstance(obj, dict):
        for key, value in obj.items():
            new_key = f"{prefix}.{key}" if prefix else key
            structure[new_key] = get_field_structure(value, new_key)
    elif isinstance(obj, list) and obj:
        structure[prefix] = get_field_structure(obj[0], prefix)
    else:
        structure[prefix] = type(obj).__name__
    return structure

def generate_readme(spark, parquet_root_directory, output_file):
    # Initialize the content of the README
    readme_content = "# FHIR Resources in Parquet Format\n\n"
    readme_content += "This document provides an overview of the FHIR resources stored in Parquet format.\n\n"
    readme_content += "## Schema Information\n\n"
    readme_content += "All resources follow the same schema:\n\n"
    readme_content += "```json\n"
    readme_content += '{"fields":[{"metadata":{},"name":"resource","nullable":true,"type":"string"}],"type":"struct"}\n'
    readme_content += "```\n\n"
    readme_content += "Each Parquet file contains a single column named `resource` of type `string`, "
    readme_content += "which stores the entire FHIR resource as a JSON string.\n\n"
    readme_content += "## Resources\n\n"

    # Get all directories in the root folder (each directory is a Parquet file)
    parquet_folders = [f for f in os.listdir(parquet_root_directory) if os.path.isdir(os.path.join(parquet_root_directory, f))]

    # Sort folders alphabetically
    parquet_folders.sort()

    for folder in parquet_folders:
        folder_path = os.path.join(parquet_root_directory, folder)
        
        # Read the Parquet file (folder)
        df = spark.read.parquet(folder_path)
        
        # Add resource information to README
        readme_content += f"### {folder}\n\n"
        
        # Get the first row to analyze structure
        sample_row = df.limit(1).collect()
        if sample_row:
            sample_json = json.loads(sample_row[0]['resource'])
            
            # Get and display field structure
            readme_content += "#### Field Structure\n\n"
            readme_content += "```\n"
            structure = get_field_structure(sample_json)
            for key, value in structure.items():
                readme_content += f"{key}: {value}\n"
            readme_content += "```\n\n"
        
            # Extract and display key FHIR fields
            readme_content += "#### Key Fields\n\n"
            key_fields = ['resourceType', 'id', 'status']
            for field in key_fields:
                if field in sample_json:
                    value = sample_json[field]
                    if isinstance(value, dict):
                        value = json.dumps(value)
                    readme_content += f"- **{field}**: {value}\n"
            readme_content += "\n"
        
        # Add some statistics
        row_count = df.count()
        readme_content += f"#### Statistics\n\n"
        readme_content += f"- Number of resources: {row_count}\n\n"

    # Write the README file
    with open(output_file, 'w') as f:
        f.write(readme_content)

    print(f"README file has been generated: {output_file}")
    
    # Display the content in the notebook
    display(Markdown(readme_content))

# Generate the README
generate_readme(spark, parquet_root_directory, output_file)

# Stop the Spark session
spark.stop()

README file has been generated: /home/jovyan/work/fhir-data/README.md


# FHIR Resources in Parquet Format

This document provides an overview of the FHIR resources stored in Parquet format.

## Schema Information

All resources follow the same schema:

```json
{"fields":[{"metadata":{},"name":"resource","nullable":true,"type":"string"}],"type":"struct"}
```

Each Parquet file contains a single column named `resource` of type `string`, which stores the entire FHIR resource as a JSON string.

## Resources

### claims

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
status: {'status': 'str'}
type: {'type.coding': {'type.coding': {'type.coding.system': {'type.coding.system': 'str'}, 'type.coding.code': {'type.coding.code': 'str'}}}}
use: {'use': 'str'}
patient: {'patient.reference': {'patient.reference': 'str'}, 'patient.display': {'patient.display': 'str'}}
billablePeriod: {'billablePeriod.start': {'billablePeriod.start': 'str'}, 'billablePeriod.end': {'billablePeriod.end': 'str'}}
created: {'created': 'str'}
provider: {'provider.reference': {'provider.reference': 'str'}, 'provider.display': {'provider.display': 'str'}}
priority: {'priority.coding': {'priority.coding': {'priority.coding.system': {'priority.coding.system': 'str'}, 'priority.coding.code': {'priority.coding.code': 'str'}}}}
facility: {'facility.reference': {'facility.reference': 'str'}, 'facility.display': {'facility.display': 'str'}}
supportingInfo: {'supportingInfo': {'supportingInfo.sequence': {'supportingInfo.sequence': 'int'}, 'supportingInfo.category': {'supportingInfo.category.coding': {'supportingInfo.category.coding': {'supportingInfo.category.coding.system': {'supportingInfo.category.coding.system': 'str'}, 'supportingInfo.category.coding.code': {'supportingInfo.category.coding.code': 'str'}}}}, 'supportingInfo.valueReference': {'supportingInfo.valueReference.reference': {'supportingInfo.valueReference.reference': 'str'}}}}
insurance: {'insurance': {'insurance.sequence': {'insurance.sequence': 'int'}, 'insurance.focal': {'insurance.focal': 'bool'}, 'insurance.coverage': {'insurance.coverage.display': {'insurance.coverage.display': 'str'}}}}
item: {'item': {'item.sequence': {'item.sequence': 'int'}, 'item.productOrService': {'item.productOrService.coding': {'item.productOrService.coding': {'item.productOrService.coding.system': {'item.productOrService.coding.system': 'str'}, 'item.productOrService.coding.code': {'item.productOrService.coding.code': 'str'}, 'item.productOrService.coding.display': {'item.productOrService.coding.display': 'str'}}}, 'item.productOrService.text': {'item.productOrService.text': 'str'}}, 'item.encounter': {'item.encounter': {'item.encounter.reference': {'item.encounter.reference': 'str'}}}}}
total: {'total.value': {'total.value': 'float'}, 'total.currency': {'total.currency': 'str'}}
```

#### Key Fields

- **resourceType**: Claim
- **id**: a8b72fa0-f930-bbba-972d-f1a9adca8456
- **status**: active

#### Statistics

- Number of resources: 344

### conditions

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
clinicalStatus: {'clinicalStatus.coding': {'clinicalStatus.coding': {'clinicalStatus.coding.system': {'clinicalStatus.coding.system': 'str'}, 'clinicalStatus.coding.code': {'clinicalStatus.coding.code': 'str'}}}}
verificationStatus: {'verificationStatus.coding': {'verificationStatus.coding': {'verificationStatus.coding.system': {'verificationStatus.coding.system': 'str'}, 'verificationStatus.coding.code': {'verificationStatus.coding.code': 'str'}}}}
category: {'category': {'category.coding': {'category.coding': {'category.coding.system': {'category.coding.system': 'str'}, 'category.coding.code': {'category.coding.code': 'str'}, 'category.coding.display': {'category.coding.display': 'str'}}}}}
code: {'code.coding': {'code.coding': {'code.coding.system': {'code.coding.system': 'str'}, 'code.coding.code': {'code.coding.code': 'str'}, 'code.coding.display': {'code.coding.display': 'str'}}}, 'code.text': {'code.text': 'str'}}
subject: {'subject.reference': {'subject.reference': 'str'}}
encounter: {'encounter.reference': {'encounter.reference': 'str'}}
onsetDateTime: {'onsetDateTime': 'str'}
recordedDate: {'recordedDate': 'str'}
```

#### Key Fields

- **resourceType**: Condition
- **id**: 3493a675-5a2f-aaf4-686a-56dc90aa96c3

#### Statistics

- Number of resources: 33

### diagnosticreports

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
status: {'status': 'str'}
category: {'category': {'category.coding': {'category.coding': {'category.coding.system': {'category.coding.system': 'str'}, 'category.coding.code': {'category.coding.code': 'str'}, 'category.coding.display': {'category.coding.display': 'str'}}}}}
code: {'code.coding': {'code.coding': {'code.coding.system': {'code.coding.system': 'str'}, 'code.coding.code': {'code.coding.code': 'str'}, 'code.coding.display': {'code.coding.display': 'str'}}}}
subject: {'subject.reference': {'subject.reference': 'str'}}
encounter: {'encounter.reference': {'encounter.reference': 'str'}}
effectiveDateTime: {'effectiveDateTime': 'str'}
issued: {'issued': 'str'}
performer: {'performer': {'performer.reference': {'performer.reference': 'str'}, 'performer.display': {'performer.display': 'str'}}}
presentedForm: {'presentedForm': {'presentedForm.contentType': {'presentedForm.contentType': 'str'}, 'presentedForm.data': {'presentedForm.data': 'str'}}}
```

#### Key Fields

- **resourceType**: DiagnosticReport
- **id**: 4f0f99cc-1402-c6a0-80e6-ea5767018811
- **status**: final

#### Statistics

- Number of resources: 344

### documentreferences

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
status: {'status': 'str'}
type: {'type.coding': {'type.coding': {'type.coding.system': {'type.coding.system': 'str'}, 'type.coding.code': {'type.coding.code': 'str'}, 'type.coding.display': {'type.coding.display': 'str'}}}}
category: {'category': {'category.coding': {'category.coding': {'category.coding.system': {'category.coding.system': 'str'}, 'category.coding.code': {'category.coding.code': 'str'}, 'category.coding.display': {'category.coding.display': 'str'}}}}}
subject: {'subject.reference': {'subject.reference': 'str'}}
date: {'date': 'str'}
author: {'author': {'author.reference': {'author.reference': 'str'}, 'author.display': {'author.display': 'str'}}}
custodian: {'custodian.reference': {'custodian.reference': 'str'}, 'custodian.display': {'custodian.display': 'str'}}
content: {'content': {'content.attachment': {'content.attachment.contentType': {'content.attachment.contentType': 'str'}, 'content.attachment.data': {'content.attachment.data': 'str'}}, 'content.format': {'content.format.system': {'content.format.system': 'str'}, 'content.format.code': {'content.format.code': 'str'}, 'content.format.display': {'content.format.display': 'str'}}}}
context: {'context.encounter': {'context.encounter': {'context.encounter.reference': {'context.encounter.reference': 'str'}}}, 'context.period': {'context.period.start': {'context.period.start': 'str'}, 'context.period.end': {'context.period.end': 'str'}}}
```

#### Key Fields

- **resourceType**: DocumentReference
- **id**: 098af643-d7dd-37bb-ff9e-7fd292619252
- **status**: superseded

#### Statistics

- Number of resources: 344

### encounters

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
identifier: {'identifier': {'identifier.use': {'identifier.use': 'str'}, 'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
status: {'status': 'str'}
class: {'class.system': {'class.system': 'str'}, 'class.code': {'class.code': 'str'}}
type: {'type': {'type.coding': {'type.coding': {'type.coding.system': {'type.coding.system': 'str'}, 'type.coding.code': {'type.coding.code': 'str'}, 'type.coding.display': {'type.coding.display': 'str'}}}, 'type.text': {'type.text': 'str'}}}
subject: {'subject.reference': {'subject.reference': 'str'}, 'subject.display': {'subject.display': 'str'}}
participant: {'participant': {'participant.type': {'participant.type': {'participant.type.coding': {'participant.type.coding': {'participant.type.coding.system': {'participant.type.coding.system': 'str'}, 'participant.type.coding.code': {'participant.type.coding.code': 'str'}, 'participant.type.coding.display': {'participant.type.coding.display': 'str'}}}, 'participant.type.text': {'participant.type.text': 'str'}}}, 'participant.period': {'participant.period.start': {'participant.period.start': 'str'}, 'participant.period.end': {'participant.period.end': 'str'}}, 'participant.individual': {'participant.individual.reference': {'participant.individual.reference': 'str'}, 'participant.individual.display': {'participant.individual.display': 'str'}}}}
period: {'period.start': {'period.start': 'str'}, 'period.end': {'period.end': 'str'}}
location: {'location': {'location.location': {'location.location.reference': {'location.location.reference': 'str'}, 'location.location.display': {'location.location.display': 'str'}}}}
serviceProvider: {'serviceProvider.reference': {'serviceProvider.reference': 'str'}, 'serviceProvider.display': {'serviceProvider.display': 'str'}}
```

#### Key Fields

- **resourceType**: Encounter
- **id**: e9f7de5c-17ab-4a4f-3c94-9d4027aac0b4
- **status**: finished

#### Statistics

- Number of resources: 344

### explanationofbenefits

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
contained: {'contained': {'contained.resourceType': {'contained.resourceType': 'str'}, 'contained.id': {'contained.id': 'str'}, 'contained.status': {'contained.status': 'str'}, 'contained.intent': {'contained.intent': 'str'}, 'contained.subject': {'contained.subject.reference': {'contained.subject.reference': 'str'}}, 'contained.requester': {'contained.requester.reference': {'contained.requester.reference': 'str'}}, 'contained.performer': {'contained.performer': {'contained.performer.reference': {'contained.performer.reference': 'str'}}}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
status: {'status': 'str'}
type: {'type.coding': {'type.coding': {'type.coding.system': {'type.coding.system': 'str'}, 'type.coding.code': {'type.coding.code': 'str'}}}}
use: {'use': 'str'}
patient: {'patient.reference': {'patient.reference': 'str'}}
billablePeriod: {'billablePeriod.start': {'billablePeriod.start': 'str'}, 'billablePeriod.end': {'billablePeriod.end': 'str'}}
created: {'created': 'str'}
insurer: {'insurer.display': {'insurer.display': 'str'}}
provider: {'provider.reference': {'provider.reference': 'str'}}
referral: {'referral.reference': {'referral.reference': 'str'}}
facility: {'facility.reference': {'facility.reference': 'str'}, 'facility.display': {'facility.display': 'str'}}
claim: {'claim.reference': {'claim.reference': 'str'}}
outcome: {'outcome': 'str'}
careTeam: {'careTeam': {'careTeam.sequence': {'careTeam.sequence': 'int'}, 'careTeam.provider': {'careTeam.provider.reference': {'careTeam.provider.reference': 'str'}}, 'careTeam.role': {'careTeam.role.coding': {'careTeam.role.coding': {'careTeam.role.coding.system': {'careTeam.role.coding.system': 'str'}, 'careTeam.role.coding.code': {'careTeam.role.coding.code': 'str'}, 'careTeam.role.coding.display': {'careTeam.role.coding.display': 'str'}}}}}}
insurance: {'insurance': {'insurance.focal': {'insurance.focal': 'bool'}, 'insurance.coverage': {'insurance.coverage.reference': {'insurance.coverage.reference': 'str'}, 'insurance.coverage.display': {'insurance.coverage.display': 'str'}}}}
item: {'item': {'item.sequence': {'item.sequence': 'int'}, 'item.category': {'item.category.coding': {'item.category.coding': {'item.category.coding.system': {'item.category.coding.system': 'str'}, 'item.category.coding.code': {'item.category.coding.code': 'str'}, 'item.category.coding.display': {'item.category.coding.display': 'str'}}}}, 'item.productOrService': {'item.productOrService.coding': {'item.productOrService.coding': {'item.productOrService.coding.system': {'item.productOrService.coding.system': 'str'}, 'item.productOrService.coding.code': {'item.productOrService.coding.code': 'str'}, 'item.productOrService.coding.display': {'item.productOrService.coding.display': 'str'}}}, 'item.productOrService.text': {'item.productOrService.text': 'str'}}, 'item.servicedPeriod': {'item.servicedPeriod.start': {'item.servicedPeriod.start': 'str'}, 'item.servicedPeriod.end': {'item.servicedPeriod.end': 'str'}}, 'item.locationCodeableConcept': {'item.locationCodeableConcept.coding': {'item.locationCodeableConcept.coding': {'item.locationCodeableConcept.coding.system': {'item.locationCodeableConcept.coding.system': 'str'}, 'item.locationCodeableConcept.coding.code': {'item.locationCodeableConcept.coding.code': 'str'}, 'item.locationCodeableConcept.coding.display': {'item.locationCodeableConcept.coding.display': 'str'}}}}, 'item.encounter': {'item.encounter': {'item.encounter.reference': {'item.encounter.reference': 'str'}}}}}
total: {'total': {'total.category': {'total.category.coding': {'total.category.coding': {'total.category.coding.system': {'total.category.coding.system': 'str'}, 'total.category.coding.code': {'total.category.coding.code': 'str'}, 'total.category.coding.display': {'total.category.coding.display': 'str'}}}, 'total.category.text': {'total.category.text': 'str'}}, 'total.amount': {'total.amount.value': {'total.amount.value': 'float'}, 'total.amount.currency': {'total.amount.currency': 'str'}}}}
payment: {'payment.amount': {'payment.amount.value': {'payment.amount.value': 'float'}, 'payment.amount.currency': {'payment.amount.currency': 'str'}}}
```

#### Key Fields

- **resourceType**: ExplanationOfBenefit
- **id**: b81e6474-23ac-fec0-cef1-906b58114d5f
- **status**: active

#### Statistics

- Number of resources: 344

### immunizations

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
status: {'status': 'str'}
vaccineCode: {'vaccineCode.coding': {'vaccineCode.coding': {'vaccineCode.coding.system': {'vaccineCode.coding.system': 'str'}, 'vaccineCode.coding.code': {'vaccineCode.coding.code': 'str'}, 'vaccineCode.coding.display': {'vaccineCode.coding.display': 'str'}}}, 'vaccineCode.text': {'vaccineCode.text': 'str'}}
patient: {'patient.reference': {'patient.reference': 'str'}}
encounter: {'encounter.reference': {'encounter.reference': 'str'}}
occurrenceDateTime: {'occurrenceDateTime': 'str'}
primarySource: {'primarySource': 'bool'}
location: {'location.reference': {'location.reference': 'str'}, 'location.display': {'location.display': 'str'}}
```

#### Key Fields

- **resourceType**: Immunization
- **id**: d387d0d4-3249-b6b7-a3c1-0e67e84ea6cb
- **status**: completed

#### Statistics

- Number of resources: 472

### locations

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
status: {'status': 'str'}
name: {'name': 'str'}
telecom: {'telecom': {'telecom.system': {'telecom.system': 'str'}, 'telecom.value': {'telecom.value': 'str'}}}
address: {'address.line': {'address.line': {'address.line': 'str'}}, 'address.city': {'address.city': 'str'}, 'address.state': {'address.state': 'str'}, 'address.postalCode': {'address.postalCode': 'str'}, 'address.country': {'address.country': 'str'}}
position: {'position.longitude': {'position.longitude': 'float'}, 'position.latitude': {'position.latitude': 'float'}}
managingOrganization: {'managingOrganization.identifier': {'managingOrganization.identifier.system': {'managingOrganization.identifier.system': 'str'}, 'managingOrganization.identifier.value': {'managingOrganization.identifier.value': 'str'}}, 'managingOrganization.display': {'managingOrganization.display': 'str'}}
```

#### Key Fields

- **resourceType**: Location
- **id**: c6e812b0-9e2e-3d09-b110-3c364f2ef2ea
- **status**: active

#### Statistics

- Number of resources: 60

### organizations

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
extension: {'extension': {'extension.url': {'extension.url': 'str'}, 'extension.valueInteger': {'extension.valueInteger': 'int'}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
active: {'active': 'bool'}
type: {'type': {'type.coding': {'type.coding': {'type.coding.system': {'type.coding.system': 'str'}, 'type.coding.code': {'type.coding.code': 'str'}, 'type.coding.display': {'type.coding.display': 'str'}}}, 'type.text': {'type.text': 'str'}}}
name: {'name': 'str'}
telecom: {'telecom': {'telecom.system': {'telecom.system': 'str'}, 'telecom.value': {'telecom.value': 'str'}}}
address: {'address': {'address.line': {'address.line': {'address.line': 'str'}}, 'address.city': {'address.city': 'str'}, 'address.state': {'address.state': 'str'}, 'address.postalCode': {'address.postalCode': 'str'}, 'address.country': {'address.country': 'str'}}}
```

#### Key Fields

- **resourceType**: Organization
- **id**: b574fc4f-313c-38a4-9baf-7e12e431a499

#### Statistics

- Number of resources: 59

### patients

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
text: {'text.status': {'text.status': 'str'}, 'text.div': {'text.div': 'str'}}
extension: {'extension': {'extension.url': {'extension.url': 'str'}, 'extension.extension': {'extension.extension': {'extension.extension.url': {'extension.extension.url': 'str'}, 'extension.extension.valueCoding': {'extension.extension.valueCoding.system': {'extension.extension.valueCoding.system': 'str'}, 'extension.extension.valueCoding.code': {'extension.extension.valueCoding.code': 'str'}, 'extension.extension.valueCoding.display': {'extension.extension.valueCoding.display': 'str'}}}}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
name: {'name': {'name.use': {'name.use': 'str'}, 'name.family': {'name.family': 'str'}, 'name.given': {'name.given': {'name.given': 'str'}}, 'name.prefix': {'name.prefix': {'name.prefix': 'str'}}}}
telecom: {'telecom': {'telecom.system': {'telecom.system': 'str'}, 'telecom.value': {'telecom.value': 'str'}, 'telecom.use': {'telecom.use': 'str'}}}
gender: {'gender': 'str'}
birthDate: {'birthDate': 'str'}
address: {'address': {'address.extension': {'address.extension': {'address.extension.url': {'address.extension.url': 'str'}, 'address.extension.extension': {'address.extension.extension': {'address.extension.extension.url': {'address.extension.extension.url': 'str'}, 'address.extension.extension.valueDecimal': {'address.extension.extension.valueDecimal': 'float'}}}}}, 'address.line': {'address.line': {'address.line': 'str'}}, 'address.city': {'address.city': 'str'}, 'address.state': {'address.state': 'str'}, 'address.postalCode': {'address.postalCode': 'str'}, 'address.country': {'address.country': 'str'}}}
maritalStatus: {'maritalStatus.coding': {'maritalStatus.coding': {'maritalStatus.coding.system': {'maritalStatus.coding.system': 'str'}, 'maritalStatus.coding.code': {'maritalStatus.coding.code': 'str'}, 'maritalStatus.coding.display': {'maritalStatus.coding.display': 'str'}}}, 'maritalStatus.text': {'maritalStatus.text': 'str'}}
multipleBirthBoolean: {'multipleBirthBoolean': 'bool'}
communication: {'communication': {'communication.language': {'communication.language.coding': {'communication.language.coding': {'communication.language.coding.system': {'communication.language.coding.system': 'str'}, 'communication.language.coding.code': {'communication.language.coding.code': 'str'}, 'communication.language.coding.display': {'communication.language.coding.display': 'str'}}}, 'communication.language.text': {'communication.language.text': 'str'}}}}
```

#### Key Fields

- **resourceType**: Patient
- **id**: 5d193e66-0268-6e9b-516d-be8c914d92da

#### Statistics

- Number of resources: 33

### practitionerroles

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
practitioner: {'practitioner.identifier': {'practitioner.identifier.system': {'practitioner.identifier.system': 'str'}, 'practitioner.identifier.value': {'practitioner.identifier.value': 'str'}}, 'practitioner.display': {'practitioner.display': 'str'}}
organization: {'organization.identifier': {'organization.identifier.system': {'organization.identifier.system': 'str'}, 'organization.identifier.value': {'organization.identifier.value': 'str'}}, 'organization.display': {'organization.display': 'str'}}
code: {'code': {'code.coding': {'code.coding': {'code.coding.system': {'code.coding.system': 'str'}, 'code.coding.code': {'code.coding.code': 'str'}, 'code.coding.display': {'code.coding.display': 'str'}}}, 'code.text': {'code.text': 'str'}}}
specialty: {'specialty': {'specialty.coding': {'specialty.coding': {'specialty.coding.system': {'specialty.coding.system': 'str'}, 'specialty.coding.code': {'specialty.coding.code': 'str'}, 'specialty.coding.display': {'specialty.coding.display': 'str'}}}, 'specialty.text': {'specialty.text': 'str'}}}
location: {'location': {'location.identifier': {'location.identifier.system': {'location.identifier.system': 'str'}, 'location.identifier.value': {'location.identifier.value': 'str'}}, 'location.display': {'location.display': 'str'}}}
telecom: {'telecom': {'telecom.system': {'telecom.system': 'str'}, 'telecom.value': {'telecom.value': 'str'}}}
```

#### Key Fields

- **resourceType**: PractitionerRole
- **id**: 456b00cb-589b-3e45-9c23-56879d9fbf70

#### Statistics

- Number of resources: 59

### practitioners

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
extension: {'extension': {'extension.url': {'extension.url': 'str'}, 'extension.valueInteger': {'extension.valueInteger': 'int'}}}
identifier: {'identifier': {'identifier.system': {'identifier.system': 'str'}, 'identifier.value': {'identifier.value': 'str'}}}
active: {'active': 'bool'}
name: {'name': {'name.family': {'name.family': 'str'}, 'name.given': {'name.given': {'name.given': 'str'}}, 'name.prefix': {'name.prefix': {'name.prefix': 'str'}}}}
telecom: {'telecom': {'telecom.extension': {'telecom.extension': {'telecom.extension.url': {'telecom.extension.url': 'str'}, 'telecom.extension.valueBoolean': {'telecom.extension.valueBoolean': 'bool'}}}, 'telecom.system': {'telecom.system': 'str'}, 'telecom.value': {'telecom.value': 'str'}, 'telecom.use': {'telecom.use': 'str'}}}
address: {'address': {'address.line': {'address.line': {'address.line': 'str'}}, 'address.city': {'address.city': 'str'}, 'address.state': {'address.state': 'str'}, 'address.postalCode': {'address.postalCode': 'str'}, 'address.country': {'address.country': 'str'}}}
gender: {'gender': 'str'}
```

#### Key Fields

- **resourceType**: Practitioner
- **id**: 6d123662-ac9b-3f2c-aa64-543a8891e476

#### Statistics

- Number of resources: 59

### provenances

#### Field Structure

```
resourceType: {'resourceType': 'str'}
id: {'id': 'str'}
meta: {'meta.profile': {'meta.profile': {'meta.profile': 'str'}}}
target: {'target': {'target.reference': {'target.reference': 'str'}}}
recorded: {'recorded': 'str'}
agent: {'agent': {'agent.type': {'agent.type.coding': {'agent.type.coding': {'agent.type.coding.system': {'agent.type.coding.system': 'str'}, 'agent.type.coding.code': {'agent.type.coding.code': 'str'}, 'agent.type.coding.display': {'agent.type.coding.display': 'str'}}}, 'agent.type.text': {'agent.type.text': 'str'}}, 'agent.who': {'agent.who.reference': {'agent.who.reference': 'str'}, 'agent.who.display': {'agent.who.display': 'str'}}, 'agent.onBehalfOf': {'agent.onBehalfOf.reference': {'agent.onBehalfOf.reference': 'str'}, 'agent.onBehalfOf.display': {'agent.onBehalfOf.display': 'str'}}}}
```

#### Key Fields

- **resourceType**: Provenance
- **id**: 160cb5d0-b1c0-69f3-04f7-861f5b88d1a5

#### Statistics

- Number of resources: 33

